In [28]:
heading = '''<div style="border-top: 5px solid #000;
border-bottom: 1px solid #000;
padding-top: 8px;
padding-bottom: 8px;
margin-bottom: 24px;
line-height: 1.4;" data-test="title">
    <h2 class="u-ma-0">{}</h2>
</div>'''

style = '''<head><style>
    .u-full-height.c-card.c-card--flush{
    display:inline-flex;
    flex-flow:wrap;
    }
    li.app-article-list-row__item{display:flex;border:solid}
    div.c-card__layout.u-full-height {
        width: 100%;
        justify-content: space-between;
        display: flex;
        flex-direction: row-reverse;
    }
    div.c-card__section.c-meta {flex: 0 0 230px;}
  </style></head>'''

import requests
from bs4 import BeautifulSoup
from bs4.element import Tag
from datetime import datetime
def write_html(date = datetime.today()):
    class Article:
        def __init__(self,html:Tag) -> None:
            name_href = html.find("a",{"class": "c-card__link u-link-inherit"})
            self.name = name_href.text if name_href != None else ""
            self.url = "https://www.nature.com"+ name_href.attrs["href"] if name_href != None else ""
            self.date = datetime.strptime(html.find("time").attrs["datetime"],'%Y-%m-%d')
            self.magazine = html.find("span",{"class": "c-meta__type"}).text
            self.html = html
            self.url = name_href.get("href")
            pass

    journals = []
    for journal in list((x if not x.startswith("#") else 0 for x in open("MagToScan.txt","r",).readlines())):
        if journal.startswith("$"):
            journals.append(journal[1:].removesuffix("\n"))

    all_magazines = [r"https://www.nature.com"+journal+r"research-articles" for journal in journals]
    articles_by_mags = {}
    len_maga = len(all_magazines)
    failed = []
    for num,magazine in enumerate(all_magazines):
        try:
            mag_soup = BeautifulSoup(requests.get(magazine).content, 'html.parser')
            articles_page = mag_soup.find_all("li",{"class": "app-article-list-row__item"})
            articles = [Article(article) for article in articles_page]
            articles_by_mags.update({magazine.split("/")[-2]: articles})
            print(f"Scanned {num+1} out of {len_maga} magazines",end="\r")
        except:
            print(f"Error with {magazine}")
            failed.append(magazine)
            pass


    base_url = "https://www.nature.com"
    with open("index.html", "w+", encoding="utf-8") as file:
        file.write("")

    with open("index.html", "a+", encoding="utf-8") as file:
        file.write("<html>")
        file.write(style)
        file.write("<body>")
        for num, (mag, articles) in enumerate(articles_by_mags.items()):
            filtered_articles = list(filter(lambda x: x.date.date() >= date.date(), articles))
            for num, article in enumerate(filtered_articles):
                html_with_absolute_hrefs = str(article.html).replace('href="/', f'href="{base_url}/')
                file.write(html_with_absolute_hrefs)
        file.write("</body></html>")

write_html()

In [36]:
requests.get("https://www.nature.com/npjpcrm/research-articles")

<Response [200]>

In [21]:
article

NameError: name 'article' is not defined

['/cmi/',
 '/gt/',
 '/gene/',
 '/jhg/',
 '/mp/',
 '/nature/',
 '/nataging/',
 '/nbt/',
 '/nchembio/',
 '/natmachintell/',
 '/neuro/']